In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import os
import plotly.graph_objects as go
import plotly.express as px
import plotly
import pathlib
plotly.io.orca.config.executable = str(pathlib.Path().absolute()) + '/bins/orca'
print(plotly.io.orca.config.executable)

In [ ]:
filenames = list(glob.iglob("../OUTPUT/Simple/*.csv"))
print(len(filenames))

In [ ]:
pd_dataframes = {'continuous': [], 'phi': [], 'summary': [], 'positions': [], 'initialpositions': [], 'finished': [], 'lost': []}
for filename in filenames:
    full_path = os.path.abspath(filename)
    path = os.path.dirname(full_path)
    basename = os.path.basename(filename)
    finished_filename = path + "/" + "finished_" + "_".join(basename.split("_")[1:])
    #if not os.path.isfile(finished_filename):
    #    print("%s does not exist ! " % os.path.basename(finished_filename))
    #    print("%s does exist" % basename)
    #    continue
    try:
        data_file_delimiter = ','
        header = []
        with open(full_path, 'r') as temp_f:
            header = temp_f.readline().rstrip('\n').split(data_file_delimiter) + ['empty']
        df = pd.read_csv(full_path, names=header, skiprows=1)
        basename = os.path.basename(filename)
        name = basename.split("_")[0]
        arguments = basename.rstrip(".csv").split("_")[3:]
        it = iter(arguments)
        for val in it:
            parameter = val.split("=")[0]
            value = val.split("=")[1]
            df[parameter] = value
        pd_dataframes[name].append(df)
    except Exception as e:
        print(e)
        #print(full_path)
        print("Error: "+ full_path)

In [ ]:
dfo = {}
for name in pd_dataframes:
    if(pd_dataframes[name]):
        dfo[name] = pd.concat(pd_dataframes[name])
df = dfo.copy()

In [ ]:
df['continuous']['Throughput'] = (df['continuous']['Rx Bytes']*8)

In [ ]:
df['continuous']

In [ ]:
common_columns = ['N', 'distance', 'wifiManager', 'simulationTime', 'antennaGain', 'agentType', 'dataRate', 'udp', 'RngRun']
df['positions'] = df['positions'].set_index(common_columns)
new_df = df['continuous'].join(df['positions'], how='left', on=common_columns, lsuffix='_left', rsuffix='_right')
df['summary'] = df['summary'].set_index(common_columns + ['Flow', 'Source Address', 'Destination Address'])
new_df = new_df.join(df['summary'], how='left', on=common_columns + ['Flow', 'Source Address', 'Destination Address'], rsuffix='_total')

In [ ]:
convergence_time = []
g = None
for name, group in new_df.groupby(common_columns + ['Flow']):
    antenna_convergence = group['last_change'].mean()
    throughput_convergence = None
    full_rx = None
    g = group
    for agent_id, agent_group in group.groupby(['agent_id']):
        if agent_id == 0:
            low, high = 0.95*(agent_group['Rx Bytes'].values[-1]), 1.05*(agent_group['Rx Bytes'].values[-1])
            agent_group['between'] = agent_group['Rx Bytes'].between(low, high)
            agent_group['between'] = ~agent_group['between']
            k = agent_group['between'][agent_group['between']].index.max()
            #k = agent_group['between'][agent_group['between']==False].last_valid_index()
            #print(k)
            if (k is None) or (pd.isna(k)):
                throughput_convergence = 0
            else:
                throughput_convergence = agent_group.loc[k]['Time']
            if name[4] == "Drone":
                full_rx = agent_group[agent_group['Time'] >= antenna_convergence]['Throughput'].mean()
            else:
                full_rx = agent_group['Throughput'].mean()
    convergence_time.append(list(name) + [throughput_convergence, antenna_convergence, full_rx])

In [ ]:
conv = pd.DataFrame(convergence_time, columns=common_columns + ['Flow', 'Throughput Converge', 'Antenna Converge', 'Throughput'])
full = conv.copy()
description = conv.groupby(['wifiManager']).describe()

# Comparaison du temps de convergence - Scénario Simple

In [ ]:
# import plotly.express as px

distance = '100'
filtered = conv[(conv['distance'] == distance) & (conv['antennaGain'] == "3.0") & (conv['agentType'] == "Drone")]

trace0 = go.Box(
    y=filtered['Antenna Converge'],
    x=filtered['wifiManager'],
    name='Orientation Convergence Time',
    boxmean=True
)

trace1 = go.Box(
    y=filtered['Throughput Converge'],
    x=filtered['wifiManager'],
    name='Throughput Convergence Time',
    boxmean=True
)

data = [trace0, trace1]
layout = go.Layout(
    #title = "foo",
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Time (s)',
        range=[-5, max(filtered['Throughput Converge'].max()*1.05, filtered['Antenna Converge'].max()*1.05)],
        zeroline=False
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("convergence_simple_3dB.png")

# Comparaison du débit final

## 3dB

In [ ]:
radius = '100'
filtered = conv[(conv['distance'] == distance) & (conv['antennaGain'] == "3.0")]
print(set(filtered['N']))
filtered = filtered[(filtered['N'] == '2')]
print(set(filtered['dataRate']))
filtered = filtered[(filtered['dataRate'] == '180Mbps')]
drone = filtered[(filtered['agentType'] == 'Drone')]
client = filtered[(filtered['agentType'] == 'Client')]

trace0 = go.Box(
    y=(client['Throughput']),
    x=client['wifiManager'],
    name='Client',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Throughput']),
    x=drone['wifiManager'],
    name='Drone',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Throughput (bps)',
    ),
    boxmode='group',)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("débit_simple_3dB.png")

In [ ]:
i = 0

for name, group in df['continuous'].groupby(['N', 'distance', 'dataRate', 'RngRun', 'antennaGain']):
    fig = go.Figure()
    for wifiManager in sorted(set(group['wifiManager'])):
        for agentType in sorted(set(group['agentType'])):
            local = group[(group['wifiManager'] == wifiManager) & (group['agentType'] == agentType)]
            str_group = name + (wifiManager, agentType)
            fig.add_trace(go.Scatter(x=local["Time"], y=local["Rx Bytes"]*8, name=" ".join(str_group)))
    fig.update_layout(
        title="", template="simple_white"
    )
    fig.update_layout(legend_orientation="h")
    fig.update_xaxes(title="Time (s)", showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(title="Throughput (bps)", showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.show()
    if (i == 3):
        print("Writing the figure")
        fig.write_image("evolution.png")
        break
    i += 1

# FIN